In [16]:
import pandas as pd
import numpy as np

In [17]:
data = pd.read_csv('adult.csv')

In [18]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split

In [19]:
enc = LabelEncoder()

In [20]:
enc.fit_transform(data['marital.status'])

array([6, 6, 6, ..., 2, 6, 4])

In [21]:
enc_df = dict()

In [22]:
data

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
32557,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32558,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32559,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K


In [23]:
for i in np.array([i for i in data.columns if data[i].dtype != 'int64']):
    enc = LabelEncoder()
    enc = enc.fit(data[i])
    enc_df[f'{i}_enc'] = enc

In [24]:
enc_df

{'workclass_enc': LabelEncoder(),
 'education_enc': LabelEncoder(),
 'marital.status_enc': LabelEncoder(),
 'occupation_enc': LabelEncoder(),
 'relationship_enc': LabelEncoder(),
 'race_enc': LabelEncoder(),
 'sex_enc': LabelEncoder(),
 'native.country_enc': LabelEncoder(),
 'income_enc': LabelEncoder()}

In [25]:
for i in np.array([i for i in data.columns if data[i].dtype != 'int64']):
    data[i] = enc_df[f'{i}_enc'].transform(data[i])

In [26]:
data

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,0,77053,11,9,6,0,1,4,0,0,4356,40,39,0
1,82,4,132870,11,9,6,4,1,4,0,0,4356,18,39,0
2,66,0,186061,15,10,6,0,4,2,0,0,4356,40,39,0
3,54,4,140359,5,4,0,7,4,4,0,0,3900,40,39,0
4,41,4,264663,15,10,5,10,3,4,0,0,3900,40,39,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,4,310152,15,10,4,11,1,4,1,0,0,40,39,0
32557,27,4,257302,7,12,2,13,5,4,0,0,0,38,39,0
32558,40,4,154374,11,9,2,7,0,4,1,0,0,40,39,1
32559,58,4,151910,11,9,6,1,4,4,0,0,0,40,39,0


In [27]:
cols = data.corr()['income'].abs().sort_values(ascending = False).head(9).index

In [28]:
cols

Index(['income', 'education.num', 'relationship', 'age', 'hours.per.week',
       'capital.gain', 'sex', 'marital.status', 'capital.loss'],
      dtype='object')

In [29]:
data.corr()['income'].abs().sort_values(ascending = False).head(9)

income            1.000000
education.num     0.335154
relationship      0.250918
age               0.234037
hours.per.week    0.229689
capital.gain      0.223329
sex               0.215980
marital.status    0.199307
capital.loss      0.150526
Name: income, dtype: float64

In [30]:
y = data[cols[0]]
X = data[cols[1:]]

In [31]:
x_train, x_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.3,
                                                   shuffle = False)

In [32]:
x_train, x_test, y_train, y_test = x_train.values, x_test.values, y_train.values, y_test.values

In [33]:
m, n = data.shape


In [34]:
def init_param(x_size, n_hidden, op_labels):
    hidden_cells_size = 10
    
    # Input layer weights and biases
    W1 = np.random.rand(10, x_size) - 0.5  # shape (10, 8) if x_size is 8
    b1 = np.random.rand(10, 1) - 0.5       # shape (10, 1)

    # Hidden layer weights and biases
    W2 = np.random.rand(hidden_cells_size, 10) - 0.5  # shape (10, 10)
    b2 = np.random.rand(hidden_cells_size, 1) - 0.5   # shape (10, 1)
    
    # Array of hidden weights and biases for each hidden layer
    hidden_weights = np.array([W2 for i in range(n_hidden)])
    hidden_bias = np.array([b2 for i in range(n_hidden)])

    # Output layer weights and biases with adjusted shapes
    W3 = np.random.rand(op_labels, hidden_cells_size) - 0.5  # shape (op_labels, 10)
    b3 = np.random.rand(op_labels, 1) - 0.5                  # shape (op_labels, 1)
    
    return W1, b1, hidden_weights, hidden_bias, W3, b3


In [35]:
def leaky_relu(z, alpha=0.01):
    return np.where(z > 0, z, alpha * z)

def deriv_leaky_relu(z, alpha=0.01):
    return np.where(z > 0, 1, alpha)

In [36]:
def relu(z):
    return np.maximum(z,0)

# def softmax(Z):
# #     shift_x = x - np.max(x)
# #     exp_x = np.exp(shift_x)
# #     a = exp_x / np.sum(exp_x)
# #     return a
#     A = np.exp(Z) / sum(np.exp(Z))
#     return A

def softmax(Z):
    shift_Z = Z - np.max(Z, axis=0, keepdims=True)  # Avoid overflow
    exp_Z = np.exp(shift_Z)
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)



In [37]:
def for_prop(x, w1, b1, hidden_weights, hidden_bias, w3, b3):
    z1 = w1.dot(x) + b1
    a1 = relu(z1)
    
    z2 = hidden_weights[0].dot(a1) + hidden_bias[0]
    a2 = relu(z2)
    
    hidden_z_list = np.array([z2 for i in range(len(hidden_bias))])
    hidden_a_list = np.array([a2 for i in range(len(hidden_bias))])
    
    for i in range(len(hidden_z_list)-1):
        hidden_z_list[i+1] = hidden_weights[i+1].dot(hidden_a_list[i]) + hidden_bias[1+1]
        hidden_a_list[i+1] = relu(hidden_z_list[i+1])
    
    z3 = w3.dot(hidden_a_list[-1]) + b3  # b3 is broadcast to match z3's shape
    a3 = softmax(z3)
    
    return z1, a1, hidden_z_list, hidden_a_list, z3, a3

In [38]:
def one_hot(y):
    one_hot_y = np.zeros((y.size, y.max()+1))
    one_hot_y[np.arange(y.size), y] = 1
    one_hot_y = one_hot_y.T
    return one_hot_y

In [39]:
def deriv_relu(z):
    return z>0

In [40]:
def back_prop(z1, a1, hidden_z_list, hidden_a_list, z3, a3, w1, hidden_weights, w3, x, y, batch_size=None):
    # Set batch size to the entire dataset if batch_size is not specified
    if batch_size is None:
        batch_size = x.shape[1]  # Use all data if no batch size is given
    
    # Slice y to match the batch size of x
    y_batch = y[:batch_size]  # Select corresponding batch from y
    one_hot_y = one_hot(y_batch)
    
    # Check and slice 'a3' if needed to match the batch size as well
    a3 = a3[:, :batch_size]
    
    # Continue with backpropagation calculations
    dz3 = a3 - one_hot_y
    m = batch_size  # for normalization by batch size

    dw3 = 1/m * dz3.dot(hidden_a_list[-1].T)
    db3 = 1/m * np.sum(dz3, axis=1, keepdims=True)

    dz2 = w3.T.dot(dz3) * deriv_relu(hidden_z_list[-1])
    dw2 = 1/m * dz2.dot(a1.T)
    db2 = 1/m * np.sum(dz2, axis=1, keepdims=True)

    hidden_dz = np.array([dz2 for i in range(len(hidden_z_list))])
    hidden_dw = np.array([dw2 for i in range(len(hidden_z_list))])
    hidden_db = np.array([db2 for i in range(len(hidden_z_list))])

    for i in range(1, len(hidden_z_list)):
        hidden_dz[-i] = hidden_weights[-i+1].T.dot(hidden_dz[-1+1]) * deriv_relu(hidden_z_list[-i])
        hidden_dw[-i] = 1/m * hidden_dz[-i].dot(hidden_a_list[-i-1].T)
        hidden_db[-i] = 1/m * np.sum(hidden_dz[-i], axis=1, keepdims=True)

    dz1 = hidden_weights[0].T.dot(hidden_dz[0]) * deriv_relu(z1)
    dw1 = 1/m * dz1.dot(x.T)
    db1 = 1/m * np.sum(dz1, axis=1, keepdims=True)
    
    return dw1, db1, hidden_dw, hidden_db, dw3, db3


In [41]:
x_train = x_train.T

In [42]:
x_test = x_test.T

In [43]:
W1, b1, hidden_weights, hidden_bias, W3, b3 = init_param(x_train.shape[0],6, 2)

In [44]:
z1, a1, hidden_z_list, hidden_a_list, z3, a3 = for_prop(x_train, W1, b1, hidden_weights, hidden_bias, W3, b3)

In [45]:
dw1, db1, hidden_dw, hidden_db, dw3, db3 = back_prop(z1, a1, hidden_z_list, hidden_a_list, z3, a3, W1, hidden_weights, W3, x_train, y_train)

In [46]:
def update_params(w1,b1,hidden_weights,hidden_bias,w3,b3,dw1, db1, hidden_dw, hidden_db, dw3, db3,lr):
    w1 = w1 - lr*dw1
    b1 = b1 - lr*db1
    for i in range(len(hidden_bias)):
        hidden_weights[i,:] = hidden_weights[i,:] - lr * hidden_dw[i]
        hidden_bias[i,:] = hidden_bias[i,:] - lr * hidden_db[i]
    
    w3 = w3 - lr*dw3
    b3 = b3 - lr*db3

    return w1, b1, hidden_weights, hidden_bias, w3, b3

In [47]:
def get_pred(a2):
    return np.argmax(a2,0)

# def get_acc(pred,y):
#     return np.sum(y == pred)/y.size

def get_acc(pred, y):
    pred = np.array(pred, dtype=int).reshape(-1)
    y = np.array(y, dtype=int).reshape(-1)
    return np.sum(y == pred) / y.size


In [48]:
def grad_desc(x,y,hidden_layers,epochs,lr):
    x_size = x.shape[0]
    n_labels = np.unique(y).shape[0]
    a = 10
    
    
    w1, b1, hidden_weights, hidden_bias, w3, b3 = init_param(x_size=x_size, n_hidden= hidden_layers, op_labels=n_labels)
    
    for i in range(epochs):
        z1, a1, hidden_z_list, hidden_a_list, z3, a3 = for_prop(x, w1, b1, hidden_weights, hidden_bias, w3, b3)
        dw1, db1, hidden_dw, hidden_db, dw3, db3 = back_prop(z1, a1, hidden_z_list, hidden_a_list, z3, a3,w1,hidden_weights,w3,x,y)
        w1, b1, hidden_weights, hidden_bias, w3, b3 = update_params(w1, b1, hidden_weights, hidden_bias, w3, b3,dw1, db1, hidden_dw, hidden_db, dw3, db3,lr)
        
        print(f'Accuracy of {i}th iteration is {get_acc(get_pred(a3),y)}')
    
    return w1, b1, hidden_weights, hidden_bias, w3, b3
#     return x,get_pred(a2)

    

In [49]:
upd_W1, upd_b1, upd_hidden_weights, upd_hidden_bias, upd_W3, upd_b3 = grad_desc(x_train, y_train,6, 10, 0.01)

Accuracy of 0th iteration is 0.7402597402597403
Accuracy of 1th iteration is 0.7385047385047385
Accuracy of 2th iteration is 0.2614952614952615
Accuracy of 3th iteration is 0.7385047385047385
Accuracy of 4th iteration is 0.7385047385047385
Accuracy of 5th iteration is 0.2614952614952615


C:\Users\tharu\AppData\Local\Temp\ipykernel_16100\2771569162.py:13: RuntimeWarning: invalid value encountered in subtract
  shift_Z = Z - np.max(Z, axis=0, keepdims=True)  # Avoid overflow


Accuracy of 6th iteration is 0.7385047385047385
Accuracy of 7th iteration is 0.7385047385047385
Accuracy of 8th iteration is 0.7385047385047385
Accuracy of 9th iteration is 0.7385047385047385


In [50]:
grad_desc(x_test, y_test,6, 10, 0.01)

Accuracy of 0th iteration is 0.8074521445388474
Accuracy of 1th iteration is 0.8074521445388474
Accuracy of 2th iteration is 0.8074521445388474
Accuracy of 3th iteration is 0.8074521445388474
Accuracy of 4th iteration is 0.8074521445388474
Accuracy of 5th iteration is 0.8074521445388474
Accuracy of 6th iteration is 0.8074521445388474
Accuracy of 7th iteration is 0.8074521445388474
Accuracy of 8th iteration is 0.8074521445388474
Accuracy of 9th iteration is 0.8074521445388474


(array([[-0.25938711,  0.22827493, -0.14606138, -0.14995627,  0.36381573,
         -0.08407312, -0.42657986, -0.11113623],
        [ 0.2451785 ,  0.31288352,  0.07325765,  0.64504676, -0.24057051,
          0.45744757, -0.0018294 ,  0.46970788],
        [ 0.1882071 , -0.48483815,  0.38834456,  0.15058196, -0.15025462,
         -0.3610139 , -0.21818523,  0.49518742],
        [ 0.48163531, -0.35924631,  0.36429208,  0.4724287 , -0.46781022,
          0.11253506,  0.09586283, -0.19792032],
        [-0.49437753,  0.16287117, -0.23038925,  0.19172677, -0.41814606,
         -0.21367532,  0.27966078,  0.17864447],
        [-0.46976317, -0.06761796, -0.02826972,  0.12238715, -0.30484585,
         -0.32100312, -0.31973107,  0.18736233],
        [-0.0366459 , -0.23750199,  0.15166312, -0.05662198,  0.11438129,
         -0.34228462,  0.02681297,  0.27520662],
        [ 0.4302688 , -0.3753754 ,  0.2608938 ,  0.17388219,  0.06570623,
          0.16088855, -0.43798346,  0.31630671],
        [ 0.2131

In [51]:
a3

array([[9.99999142e-01, 9.99999299e-01, 9.99998651e-01, ...,
        3.33058908e-01, 2.44605729e-01, 2.23632888e-01],
       [8.58291224e-07, 7.01259243e-07, 1.34892805e-06, ...,
        6.66941092e-01, 7.55394271e-01, 7.76367112e-01]])

In [52]:
y_train

array([0, 0, 0, ..., 1, 0, 0])

In [53]:
x_train.shape, y_train.shape

((8, 22792), (22792,))

In [54]:
x_test.shape, y_test.shape

((8, 9769), (9769,))

In [55]:
def make_predictions(x, w1, b1, w2, b2, w3, b3):
    _,_,_, _, _, A3 = for_prop(x, w1, b1, w2, b2, w3, b3)
    predictions = get_pred(A3)
    return predictions

In [56]:
make_predictions(x_test[:,0], upd_W1, upd_b1, upd_hidden_weights, upd_hidden_bias, upd_W3, upd_b3)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [57]:
def test_prediction(index, w1, b1, w2, b2, w3, b3):
    current_image = x_train[:, index, None]
    prediction = make_predictions(x_train[:, index, None], w1, b1, w2, b2, w3, b3)
    label = y_train[index]
    
    print("Prediction: ", prediction)
    print("Label: ", label)
    
#     current_image = current_image.reshape((28, 28)) * 255
#     plt.gray()
#     plt.imshow(current_image, interpolation='nearest')
#     plt.show()
    
    return prediction

In [65]:
test_prediction(6,upd_W1, upd_b1, upd_hidden_weights, upd_hidden_bias, upd_W3, upd_b3)

Prediction:  [0]
Label:  0


array([0], dtype=int64)

In [66]:
enc_df['income_enc']

LabelEncoder()